In [1]:
%load_ext autoreload
%autoreload 2

In [74]:
import numpy as np
from datasets import load_dataset
from docarray import BaseDoc, DocList
from docarray.index import HnswDocumentIndex
from docarray.typing import NdArray
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

In [3]:
model_name = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [9]:
ds_dict = (
    load_dataset("pietrolesci/ag_news", "concat")["test"]
    .map(
        lambda ex: {**tokenizer(ex["text"]), "embedding": embedder.encode(ex["text"], device="cuda", batch_size=256)},
        batched=True,
    )
    .rename_columns({"label": "labels"})
)

Found cached dataset ag_news (/home/pl487/.cache/huggingface/datasets/pietrolesci___ag_news/concat/1.0.0/5ee6e111adc7a901ca734b79fbebff09d9dba91722387a794efff8d9c178a6a3)


  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [85]:
class Document(BaseDoc):
    text: str
    # input_ids: list[int]
    # attention_mask: list[int]
    labels: int
    embedding: NdArray[384]

In [86]:
dl = DocList[Document](
    [
        Document(
            text=doc["text"],
            # input_ids=doc["input_ids"],
            # attention_mask=doc["attention_mask"],
            labels=doc["labels"],
            embedding=doc["embedding"],
        )
        for doc in ds_dict.to_list()
    ]
)

In [87]:
dl[0]

📄 Document : c5d39a7 ...
╭────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────╮
│ Attribute          │ Value                                                                                      │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ text: str          │ Fears for T N pension after talks Unions represent ... (length: 161)                       │
│ labels: int        │ 2                                                                                          │
│ embedding: NdArray │ NdArray of shape (384,), dtype: float64                                                    │
╰────────────────────┴────────────────────────────────────────────────────────────────────────────────────────────╯

In [88]:
db = HnswDocumentIndex[Document](work_dir="./db")

In [95]:
db.configure(
    default_column_config={
        np.ndarray: {
            "dim": -1,
            "index": True,
            "space": "cosine",
            "max_elements": 10 ^ 10,
            "ef_construction": 200,
            "ef": 15,
            "M": 8,
            "allow_replace_deleted": True,
            "num_threads": 5,
        }
        # None: {},
    }
)

In [96]:
db.index(dl)

RuntimeError: The number of elements exceeds the specified limit

In [43]:
db.find(dl[0], search_field="embedding")

KeyError: 'embedding'

In [ ]:
datastore = PandasDatastoreForSequenceClassification()
datastore.from_dataset_dict(dataset_dict=ds_dict, tokenizer=tokenizer)

In [ ]:
datastore.input_names, datastore.target_name, datastore.on_cpu

In [ ]:
datastore.hparams

In [ ]:
datastore.label_distribution()

In [ ]:
datastore.labels

In [ ]:
datastore.label2id

In [ ]:
datastore.train_dataset()

In [ ]:
datastore.label(indices=[0, 1], round=1, validation_perc=0.5)

In [ ]:
datastore.train_dataset(), datastore.train_dataset(0)

In [ ]:
datastore.pool_dataset(), datastore.pool_dataset(0)

In [ ]:
datastore.validation_dataset(), datastore.validation_dataset(0)

In [ ]:
datastore.test_dataset()

In [ ]:
datastore.show_batch()

In [ ]:
batch = datastore.show_batch()

In [ ]:
from lightning.fabric import Fabric

In [ ]:
fabric = Fabric(accelerator="gpu")

In [ ]:
fabric.to_device(batch)